# Azure AI Agent Service with OpenAPI Specified Tools

## Verify that your TripAdvisor API works 

In [4]:
import requests

url = "https://api.content.tripadvisor.com/api/v1/location/search"
params = {
    "key": "6FA816730D164E0BB2CEF1DBE823D4E1",
    "searchQuery": "new york",
    "language": "en"
}
headers = {
    "accept": "application/json"
}

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    print(response.json())  # Print the JSON response
else:
    print(f"Error: {response.status_code}, {response.text}")

{'data': [{'location_id': '60763', 'name': 'New York City', 'address_obj': {'state': 'New York', 'country': 'United States', 'address_string': 'New York City, NY'}}, {'location_id': '91904', 'name': 'New York - New York Hotel & Casino', 'address_obj': {'street1': '3790 S Las Vegas Blvd', 'city': 'Las Vegas', 'state': 'Nevada', 'country': 'United States', 'postalcode': '89109-4338', 'address_string': '3790 S Las Vegas Blvd, Las Vegas, NV 89109-4338'}}, {'location_id': '112064', 'name': 'Warwick New York', 'address_obj': {'street1': '65 W 54th St', 'street2': '', 'city': 'New York City', 'state': 'New York', 'country': 'United States', 'postalcode': '10019', 'address_string': '65 W 54th St, New York City, NY 10019'}}, {'location_id': '28953', 'name': 'New York', 'address_obj': {'country': 'United States', 'address_string': 'NY'}}, {'location_id': '970327', 'name': 'New York Cafe', 'address_obj': {'street1': 'Erzsebet Korut 9-11', 'street2': 'Anantara New York Palace Budapest Hotel', 'cit

## Step 1: Create a project client

In [15]:
import os
import jsonref
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import OpenApiTool, OpenApiConnectionAuthDetails, OpenApiConnectionSecurityScheme


# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# At the moment, it should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customer needs to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

## Step 2: Create the OpenAPI Spec tool definition

In [16]:
with open('./tripadvisor_openapi.json', 'r') as f:
    openapi_spec = jsonref.loads(f.read())

tripadvisor_connection = project_client.connections.get(
    connection_name="tripadvisor"
)
conn_id = tripadvisor_connection.id
print(conn_id)

auth = OpenApiConnectionAuthDetails(security_scheme=OpenApiConnectionSecurityScheme(connection_id=conn_id))

openapi = OpenApiTool(name="tripadvisor", spec=openapi_spec, description="get hotel and restaurant reviews of a location", auth=auth)

/subscriptions/f08cda90-375b-4b3e-a105-4656379a94ab/resourceGroups/rg-Ziggy-ai-agent-001/providers/Microsoft.MachineLearningServices/workspaces/ziggy-ai-agent-project-001/connections/tripadvisor


## Step 3: Create an agent and a thread

In [19]:
# Create agent with OpenAPI tool and process assistant run
agent = project_client.agents.create_agent(
    model="gpt-4o",
    name="my-tripadvisor-assistant",
    instructions="You are a helpful assistant that will information using the tools provided to you.",
    tools=openapi.definitions
)
print(f"Created agent, ID: {agent.id}")

# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

Created agent, ID: asst_mCpIodil137tCbOWdxA8VoL1
Created thread, ID: thread_OtzEr5yDInILfXkqavZWifrw


## Helper Function
- Add a message to the thread
- Run the Agent
- Check the Run Status
- Display the Agent's Response


In [20]:
def run_agent(user_input):  
    # Step 4: Add a message to the thread  
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=user_input,
    )
    print(f"Created message, ID: {message.id}")

    # Step 5 & 6: Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Step 7: Display the Agent's Response
    elif run.status == 'completed':
            # Fetch all messages in the thread
            messages = project_client.agents.list_messages(thread_id=thread.id)
            if messages.data:
                agent_message = messages.data[0]  # Get the last assistant message
                print(f"Agent Response: {agent_message.content[0].text.value}") 
            else:
                print("No messages found.")

## Running the agent using the Helper Function

In [21]:
user_input = "Where is the La Grande Boucherie located?"
run_agent(user_input)

Created message, ID: msg_nJjNrKBGAYiBHjsGu9qE1YGy
Agent Response: I encountered an error trying to retrieve the information about La Grande Boucherie's location. Can you provide more details or clarify further to refine the search?


In [22]:
user_input = "What is the review of the La Grande Boucherie?"
run_agent(user_input)

Created message, ID: msg_r8a7u5tgiV5PrNi2MkyEpq4H
Agent Response: I encountered an issue retrieving reviews for La Grande Boucherie. Could you provide more details or specifics to help refine the search?


## Delete Agent to free up resources

In [23]:
# Delete the agent when done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

Deleted agent
